In [2]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn import tree
from random import choices


In [ ]:
fireWall = pd.read_csv('log2.csv')

fireWallXData = fireWall[['Source Port', 'Destination Port','NAT Source Port',
                         'NAT Destination Port','Bytes','Bytes Sent','Bytes Received','Packets',
                        'Elapsed Time (sec)','pkts_sent','pkts_received']]
fireWallyData = fireWall[['Action']]

X = fireWallXData.iloc[:,0:10]
y = fireWallyData.iloc[:,0]


print(y.shape)
print(X.shape)

#print(fireWall.shape)

FireWALLData = pd.concat([X, y], axis=1)
print(FireWALLData.shape)

newX = FireWALLData.iloc[:,0:9]
newy = FireWALLData.iloc[:,10]

from sklearn.preprocessing import LabelEncoder
encodeLabel = LabelEncoder()
newy = encodeLabel.fit_transform(newy)
print(newy)


FireWALLData.loc[FireWALLData['Action'] == 0, 'Action'] = 1
FireWALLData.loc[FireWALLData['Action'] == 1, 'Action'] = 0
FireWALLData.loc[FireWALLData['Action'] == 2, 'Action'] = 1
FireWALLData.loc[FireWALLData['Action'] == 3, 'Action'] = 0

newy = FireWALLData.iloc[:,10]

FireWALLData.head()




In [ ]:


X_train, X_test, y_train, y_test = train_test_split( newX, newy, test_size=60000, random_state=1738)


# SUPPORT VECTOR MACHINE TRIAL ONE ON Fire Wall  DATASET

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)    
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_1_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

In [ ]:
TRIAL_1_SVM.cv_results_

In [ ]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']) ]

In [ ]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_f1_micro']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_f1_micro']) ]

In [ ]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

In [ ]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1_SVM.cv_results_['params'][ np.argmin(TRIAL_1_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


# SUPPORT VECTOR MACHINE TRIAL TWO ON Fire Wall  DATASET

In [ ]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', MinMaxScaler(feature_range = (0, 1))),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

In [ ]:
TRIAL_2_SVM.cv_results_

In [ ]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']) ]

In [ ]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_f1_micro']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_f1_micro']) ]

In [ ]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

In [ ]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 2 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 2 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2_SVM.cv_results_['params'][ np.argmin(TRIAL_2_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 2 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


# SUPPORT VECTOR MACHINE TRIAL THREE ON Fire Wall  DATASET

In [ ]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

In [ ]:
TRIAL_3_SVM.cv_results_

In [ ]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']) ]

In [ ]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_f1_micro']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_f1_micro']) ]

In [ ]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

In [ ]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 3 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 3 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3_SVM.cv_results_['params'][ np.argmin(TRIAL_3_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 3 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


# SUPPORT VECTOR MACHINE TRIAL FOUR ON Fire Wall  DATASET

In [ ]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

In [ ]:
TRIAL_4_SVM.cv_results_

In [ ]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']) ]

In [ ]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_f1_micro']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_f1_micro']) ]

In [ ]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

In [ ]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 4 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 4 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4_SVM.cv_results_['params'][ np.argmin(TRIAL_4_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 4 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


# SUPPORT VECTOR MACHINE TRIAL FIVE ON Fire Wall  DATASET

In [ ]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', svm.SVC(max_iter=5000,probability=True))])

    search_space = [
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['rbf'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['sigmoid'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                
                {
                
                'classifier': [svm.SVC(max_iter=7000,probability=True)],
                'classifier__kernel': ['poly'],
                'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]},
                {
                 
                 'classifier': [svm.SVC(max_iter=7000,probability=True)],
                 'classifier__kernel': ['linear'],
                 'classifier__gamma': [1e-8,1e-7,1e-6, 1e-5, 1e-4, 1e-3,1e-2,1e-1,1e-0],
                 'classifier__C': [.0010,.010,.10, 1, 10, 100, 1000,10000,100000]}, 
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5_SVM = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

In [ ]:
TRIAL_5_SVM.cv_results_

In [ ]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']) ]

In [ ]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_f1_micro']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_f1_micro']) ]

In [ ]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']) ]

In [ ]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 5 svm using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 5 svm using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5_SVM.cv_results_['params'][ np.argmin(TRIAL_5_SVM.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 5 svm using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")
